In [2]:
import azureml.core
from azureml.core import Workspace, Model

# Inference testing function definition

In [50]:
import base64
import requests
import json 

def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

def test_inf(endpoint, key=None):
    input_json = json.dumps({
        "n_cards": 5,
        "images": True,
        "hand": get_base64_encoded_image('/home/jeremy/Documents/data/quiddler/test/IMG_4903.jpg'),
        "deck": get_base64_encoded_image('/home/jeremy/Documents/data/quiddler/train/A_1.jpg')
    })
    headers = { 'Content-Type':'application/json' }
    if key:
        headers['Authorization'] = f'Bearer {key}' # Only difference to local is to send the auth key
    predictions = requests.post(endpoint, input_json, headers = headers)
    p = predictions.json()
    print(str(p)+'\n')
    print(f'Hand:     {p[0]}')
    print(f'Deck:     {p[1]}')
    if p[2]:
        play = p[2][0]
        print(f'Score:    {play[0]}')
        print(f'Complete: {play[1]}')
        print(f'Words:    {[c[0] for c in play[2]]}')
        print(f'Pick up:  {p[2][1]}')
        print(f'Drop:     {p[2][2]}\n')
    else:
        print('No possible play for these cards')    

# Connect to the Workspace

In [4]:
# Load the workspace from the saved config file
ws = Workspace.from_config()
print(f'Azure ML version: {azureml.core.VERSION}, Workspace: {ws.name}')

Azure ML version: 1.13.0, Workspace: jbWorkspace


# Create the inference conda file

In [6]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_pip_package("azureml-defaults")
myenv.add_pip_package("icevision[all]==0.1.4")
myenv.add_pip_package("inference-schema")
myenv.add_pip_package("pygtrie")
myenv.save('source/inference-env.yml')

'source/inference-env.yml'

# Deploy locally

In [41]:
model = ws.models['quiddler_model']
print(f'Model: {model.name}, Version: {model.version}')

Model: quiddler_model, Version: 1


In [42]:
from azureml.core.webservice import LocalWebservice
from azureml.core.model import InferenceConfig

In [ ]:
# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = 'source',
                                   entry_script="score.py",
                                   conda_file="inference-env.yml")

deployment_config = LocalWebservice.deploy_configuration()

service_name = "quiddler-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

## Call the inference service

In [51]:
# Reload the service to pick up changes in the source directory
service.reload(wait=True)

Container has been successfully cleaned up.
Starting Docker container...
Docker container running.
Checking container health...
Local webservice is running at http://localhost:32779


In [52]:
local_endpoint = service.scoring_uri
print(local_endpoint)

http://localhost:32779/score


In [53]:
test_inf(local_endpoint)

['g/th/in/p/u', 'a', [[32, True, [['g/u/p', [16, 3, ['g', 'u', 'p']]], ['th/in', [16, 2, ['th', 'in']]]]], 'a', 'a']]

Hand:     g/th/in/p/u
Deck:     a
Score:    32
Complete: True
Words:    ['g/u/p', 'th/in']
Pick up:  a
Drop:     a



In [ ]:
print(service.get_logs())

## Shut it down

In [34]:
service.delete()
print("Service deleted.")

Container has been successfully cleaned up.
Service deleted.


In [35]:
!docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


# Deploy into AKS

In [37]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model
from azureml.core.compute import AksCompute

service_name = "quiddler-service2"
aks_target = AksCompute(ws,"jb-inf")
# If deploying to a cluster configured for dev/test, ensure that it was created with enough
# cores and memory to handle this deployment configuration. Note that memory is also used by
# things such as dependencies and AML components.
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
aks_service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)
#print(aks_service.get_logs())

Running.........
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy


In [38]:
primary, secondary = aks_service.get_keys()
aks_endpoint = aks_service.scoring_uri
print(aks_endpoint)

http://52.177.13.18:80/api/v1/service/quiddler-service2/score


In [39]:
test_inf(aks_endpoint, primary)

['g/th/in/p/u', 'a', [[32, True, [['g/u/p', [16, 3, ['g', 'u', 'p']]], ['th/in', [16, 2, ['th', 'in']]]]], 'a', 'a']]

Hand:     g/th/in/p/u
Deck:     a
Score:    32
Complete: True
Words:    ['g/u/p', 'th/in']
Pick up:  a
Drop:     a



In [40]:
aks_service.delete()